In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
%matplotlib inline

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv('done_data.csv')
a = pd.read_csv('a.csv')

In [3]:
X = data.drop('label', axis = 1)
y = data['label']

In [4]:
X['word_match'] = a['word_match']
X['tfidf_word_match'] = a['tfidf_word_match']
X = X.fillna(0)

In [5]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from time import time

In [6]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 15)

In [7]:
def LR():
    start = time()
    from sklearn.linear_model import LogisticRegression
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    used_time = time() - start
    print 'Time used {}'.format(used_time)
    return clf
clf_LR = LR()
clf_LR.score(X_test, y_test)

Time used 41.0776560307


0.70937940587202253

In [8]:
# print clf_LR.predict_proba(X_test)
# print clf_LR.predict(X_test)
log_loss(y_test, clf_LR.predict_proba(X_test))   

0.58288433627563419

In [9]:
def RF():
    from sklearn.ensemble import RandomForestClassifier
    start = time()
    clf = RandomForestClassifier(n_estimators=20, random_state=0)
#     n_iter_search = 30
#     random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=n_iter_search)
#     random_search.fit(X_train, y_train)
    clf.fit(X_train, y_train)
    used_time = time() - start
    print 'Time used {}'.format(used_time)
    return clf
clf_RF = RF()
clf_RF.score(X_test, y_test)

Time used 170.820140839


0.78929728660120213

In [10]:
log_loss(y_test, clf_RF.predict_proba(X_test)) 

0.49357498835134

In [11]:
def xgboost():
    from sklearn.ensemble import GradientBoostingClassifier
    start = time()
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
    clf.fit(X_train, y_train)
    used_time = time() - start
    print 'Time used {}'.format(used_time)
    return clf
clf_xgb = xgboost()
clf_xgb.score(X_test, y_test)

Time used 355.119706869


0.71248361324791609

In [12]:
log_loss(y_test, clf_xgb.predict_proba(X_test)) 

0.57249591862939964

In [13]:
def xgboost2():
    params = {}
    params['objective'] = 'binary:logistic'
    params['eval_metric'] = 'logloss'
    params['eta'] = 0.02
    params['max_depth'] = 4

    d_train = xgb.DMatrix(X_train, label=y_train)
    d_valid = xgb.DMatrix(X_test, label=y_test)

    watchlist = [(d_train, 'train'), (d_valid, 'valid')]

    bst = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=50, verbose_eval=10)
    test = xgb.DMatrix(X_test)
    test_valid = bst.predict(test)
    res = pd.Series(test_valid).map(lambda x : 1 if x >= 0.5 else 0)
    accuracy_score(y_test, res)
xgboost2()
    
#     d_test = xgb.DMatrix(x_test)
#     p_test = bst.predict(d_test)
#     sub = pd.DataFrame()
#     sub['test_id'] = df_test['test_id']
#     sub['is_duplicate'] = p_test
#     sub.to_csv('simple_xgb.csv', index=False)



[0]	train-logloss:0.689891	valid-logloss:0.689913
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[10]	train-logloss:0.663399	valid-logloss:0.663621
[20]	train-logloss:0.644135	valid-logloss:0.644367
[30]	train-logloss:0.629663	valid-logloss:0.629939
[40]	train-logloss:0.6186	valid-logloss:0.618919
[50]	train-logloss:0.609214	valid-logloss:0.609753
[60]	train-logloss:0.601371	valid-logloss:0.602113
[70]	train-logloss:0.594532	valid-logloss:0.595455
[80]	train-logloss:0.588595	valid-logloss:0.589667
[90]	train-logloss:0.58306	valid-logloss:0.584252
[100]	train-logloss:0.578336	valid-logloss:0.579626
[110]	train-logloss:0.573877	valid-logloss:0.575272
[120]	train-logloss:0.570058	valid-logloss:0.571587
[130]	train-logloss:0.566558	valid-logloss:0.568185
[140]	train-logloss:0.563363	valid-logloss:0.565091
[150]	train-logloss:0.560313	valid-logloss:0.562143
[160]	train-logloss:0.557555	va